# Reweighted loss functions for the neural flow diffusion model

## Variational diffusion models

### Definition of the forward process

Let's define the forward process

\begin{align}
    z = \alpha x + \sigma \varepsilon,
    \quad \text{where} \quad
    \alpha^2 + \sigma^2 = 1
\end{align}

Then we have the following connections:

\begin{align}
    x = \frac{z - \sigma \varepsilon}{\alpha}
    \quad \text{and} \quad
    \varepsilon = \frac{z - \alpha x}{\sigma}
\end{align}

### Signal-To-Noise Ratio (SNR)

Introduce the Signal-To-Noise Ratio (SNR)

\begin{align}
    SNR = \frac{\alpha^2}{\sigma^2}
\end{align}

Reparametrization through the gamma function

\begin{align}
    SNR = e^{-\gamma}
\end{align}

Then we can rewrite the $\alpha$ and $\sigma$ coefficients in terms of the gamma function

\begin{align}
    SNR = \frac{\alpha^2}{1 - \alpha^2} = e^{-\gamma}
    \quad \Rightarrow \quad
    \alpha^2 &= \frac{e^{-\gamma}}{1 + e^{-\gamma}} = \frac{1}{1 + e^{\gamma}} = \sigma(-\gamma) \\
    \sigma^2 &= 1 - \alpha^2 = \frac{1}{1 + e^{-\gamma}} = \sigma(\gamma)
\end{align}

### Conditional ODE and SDEs

The conditional ODE is

\begin{align}
    f = \dot{\alpha} x + \dot{\sigma} \varepsilon = \dot{\alpha} x + \frac{\dot{\sigma}}{\sigma} (z - \alpha x)
\end{align}

The conditional score function is

\begin{align}
    s = - \frac{\varepsilon}{\sigma} = \frac{\alpha x - z}{\sigma^2}
\end{align}

Combining the drift of the ODE $f$ and the score function $s$ with the volatility $g$ we can write down the conditional forward SDE

\begin{align}
    d z = f^F d t + g d w, \quad \text{where} \quad f^F = f + \frac{g^2}{2} s
\end{align}

Similarly, we can write down the conditional backward SDE

\begin{align}
    d z = f^B d t + g d \bar{w}, \quad \text{where} \quad f^B = f - \frac{g^2}{2} s
\end{align}

### Derivation of the volatility

In general, the volatility $g$ can be an arbitrary function of time $t$. However, there is one useful consideration that can help us parameterise in a more efficient way.


In diffusion models, we aim to match the distribution of trajectories of the forward and reversed processes. The reverse process is Markovian by design. Therefore, to be able to match the distributions of trajectories, the forward process should also be Markovian. TO guaranry this, we can find such a volatility $g$ that makes the forward process independent on $x$.

I don't know how to derive $g$ analytically in general case, but we can do it in case of the VDM.

\begin{align}
    f^F
    &= f + \frac{g^2}{2} s \\
    &= \dot{\alpha} x + \frac{\dot{\sigma}}{\sigma} (z - \alpha x) + \frac{g^2}{2} \frac{\alpha x - z}{\sigma^2} \\
    &= \underbrace{ \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha + \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right) }_{=0} x + \left( \frac{\dot{\sigma}}{\sigma} - \frac{g^2}{2} \frac{1}{\sigma^2} \right) z \\
\end{align}

That gives us the expression for the volatility

\begin{align}
    g^2
    &= 2 \frac{\sigma^2}{\alpha} \left( \frac{\dot{\sigma}}{\sigma} \alpha - \dot{\alpha} \right) \\
    &= 2 \sigma \dot{\sigma} - 2 \sigma^2 \frac{\dot{\alpha}}{\alpha} \\
    &= (\sigma^2)' - 2 (\log \alpha)' \sigma^2
\end{align}

We can also rewrite the volatility in terms of the gamma function

\begin{align}
    g^2
    &= (\sigma^2)' - 2 (\log \alpha)' \sigma^2 \\
    &= (\sigma^2)' - \frac{2 \alpha \dot{\alpha}}{\alpha^2} \sigma^2 \\
    &= (\sigma(\gamma))' - \frac{(\sigma(-\gamma))'}{\sigma(-\gamma)} \sigma(\gamma) \\
    &= \sigma(\gamma) \left( 1 - \sigma(\gamma) \right) \dot{\gamma} + \frac{\sigma(-\gamma) \left( 1 - \sigma(-\gamma) \right) \dot{\gamma}}{\sigma(-\gamma)} \sigma(\gamma) \\
    &= \sigma(\gamma) \dot{\gamma} \left( 1 - \sigma(\gamma) + \underbrace{1 - \sigma(-\gamma)}_{=\sigma(\gamma)} \right) \\
    &= \sigma(\gamma) \dot{\gamma}
\end{align}

To keep the volatility function general, but preserve the connection with the gamma function, we derived, we can reperametrize the volatility function as follows

\begin{align}
    g^2 = \sigma(\gamma) \dot{\gamma} \eta
\end{align}

where $\eta$ is an arbitrary non-negative function of time $t$. If we set $\eta = 1$, we will recover the Markovian volatility.

### Reverse process

We define the reverse process through prediction $\hat{x}(z,t)$ that we substitute into the conditional backward SDE:

\begin{align}
    d z = \hat{f}^B d t + g d \bar{w}, \quad \text{where} \quad \hat{f}^B(z, t) = f^B(z, t, \hat{x}(z,t))
\end{align}


### Derivation of the ELBO

We know that the ELBO of diffusion models is

\begin{align}
    \mathcal{L} = \lambda_{f^B} \left\| f^B - \hat{f}^B \right\|_2^2, \quad \text{where} \quad \lambda_{f^B} = \frac{1}{2 g^2}
\end{align}

For VDM, we can rewrite the $f^B$ as:
\begin{align}
    f^B
    &= f - \frac{g^2}{2} s \\
    &= \dot{\alpha} x + \frac{\dot{\sigma}}{\sigma} (z - \alpha x) - \frac{g^2}{2} \frac{\alpha x - z}{\sigma^2} \\
    &= \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha - \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right) x + \left( \frac{\dot{\sigma}}{\sigma} + \frac{g^2}{2} \frac{1}{\sigma^2} \right) z
\end{align}

Since the second term doesn't depend on $x$ and will cancel out in the ELBO, we can rewrite the ELBO as:

\begin{align}
    \mathcal{L} = \lambda_x \left\| x - \hat{x} \right\|_2^2
\end{align}

Let's derive the $\lambda_x$ coefficient

\begin{align}
    \lambda_x
    &= \frac{1}{2 g^2} \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha - \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right)^2 \\
    &= \frac{1}{2 g^2} \left( \frac{\alpha}{2} \frac{2 \alpha \dot{\alpha}}{\alpha^2} - \frac{\alpha}{2} \frac{2 \sigma \dot{\sigma}}{\sigma^2} - \frac{\alpha}{2} \frac{g^2}{\sigma^2} \right)^2 \\
    &= \frac{1}{2 g^2} \frac{\alpha^2}{2^2} \left( \frac{(\alpha^2)'}{\alpha^2} - \frac{(\sigma^2)'}{\sigma^2} - \frac{g^2}{\sigma^2} \right)^2 \\
    &= \frac{1}{2} \frac{\alpha^2}{\sigma^2} \frac{1}{2^2 \dot{\gamma} \eta} \left( \frac{(\alpha^2)'}{\alpha^2} - \frac{(\sigma^2)'}{\sigma^2} - \frac{\sigma^2 \dot{\gamma} \eta}{\sigma^2} \right)^2 \\
    &= \frac{1}{2} e^{-\gamma} \frac{1}{2^2 \dot{\gamma} \eta} \left( \frac{\sigma(-\gamma) \left( 1 - \sigma(-\gamma) \right) (-1) \dot{\gamma}}{\sigma(-\gamma)} - \frac{\sigma(\gamma) \left( 1 - \sigma(\gamma) \right) \dot{\gamma}}{\sigma(\gamma)} - \dot{\gamma} \eta \right)^2 \\
    &= \frac{1}{2} e^{-\gamma} \frac{1}{2^2 \dot{\gamma} \eta} \left( \big[ - \underbrace{\left( 1 - \sigma(-\gamma) \right)}_{=\sigma(\gamma)} -  1 + \sigma(\gamma) \big] \dot{\gamma} - \dot{\gamma} \eta \right)^2 \\
    &= \frac{1}{2} e^{-\gamma} \frac{1}{2^2 \dot{\gamma} \eta} \left( - \dot{\gamma} - \dot{\gamma} \eta \right)^2 \\
    &= \frac{1}{2} e^{-\gamma} \frac{ \dot{\gamma}^2 \left( 1 + \eta \right)^2 }{2^2 \dot{\gamma} \eta} \\
    &= \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta}
\end{align}

Since nothing except the last coefficient depends on function $\eta$, we can see, we can easily find the optimal $\eta$. It is $\eta = 1$. Therefore, the optimal volatility function is a Markovian volatility.

We can also find a nice connection with the SNR function, when $\eta = 1$

\begin{align}
    SNR' = (e^{-\gamma})' = -e^{-\gamma} \dot{\gamma}, \quad \lambda_x = \frac{1}{2} e^{-\gamma} \dot{\gamma} = - \frac{1}{2} SNR'
\end{align}

### Alternative formulations of the ELBO

Similarly, to formulation of the ELBO in terms of the prediction $\hat{f}^B$ or $\hat{x}$, we can rewrite the ELBO in terms of prediction of $\hat{\varepsilon}$

\begin{align}
    x = \frac{z - \sigma \varepsilon}{\alpha}
\end{align}

\begin{align}
    \mathcal{L} = \lambda_\varepsilon \left\| \varepsilon - \hat{\varepsilon} \right\|_2^2
\end{align}

\begin{align}
    \lambda_\varepsilon
    &= \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} \frac{\sigma^2}{\alpha^2} \\
    &= \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} e^{\gamma} \\
    &= \frac{1}{2} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta}
\end{align}

When $\eta = 1$, we have

\begin{align}
    \log-SNR' = -\dot{\gamma}, \quad \lambda_\varepsilon = \frac{1}{2} \dot{\gamma} = - \frac{1}{2} \log-SNR'
\end{align}

We can also rewrite the ELBO in terms of the prediction of $\hat{v}$ function (see Appendix D in [this paper](https://arxiv.org/abs/2202.00512))

\begin{align}
    x = \alpha z - \sigma v
\end{align}

\begin{align}
    \mathcal{L} = \lambda_v \left\| v - \hat{v} \right\|_2^2
\end{align}

\begin{align}
    \lambda_v
    &= \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} \sigma^2 \\
    &= \frac{1}{2} \frac{e^{-\gamma}}{1 + e^{-\gamma}} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} \\
    &= \frac{1}{2} \sigma(-\gamma) \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} \\
    &= \frac{1}{2} \alpha^2 \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta}
\end{align}

### Reweighted ELBO formulations

As we know from a lot of papers, diffusion models often have better performance when trained not with the ELBO objective, but with reweighted ELBO functions like:

\begin{align}
    \mathcal{L}_x = \left\| x - \hat{x} \right\|_2^2 = \frac{1}{\lambda_x} \mathcal{L} \quad \text{or} \quad \mathcal{L}_\varepsilon = \left\| \varepsilon - \hat{\varepsilon} \right\|_2^2 = \frac{1}{\lambda_\varepsilon} \mathcal{L}
\end{align}

But what should we do if the model predicts $\hat{x}$ and we want to train a model with $\mathcal{L}_\varepsilon$? We can simply take the $\mathcal{L}_x$ or $\mathcal{L}$ and reweight it!
\begin{align}
    \mathcal{L}_\varepsilon = \frac{\lambda_x}{\lambda_\varepsilon} \mathcal{L}_x = \frac{1}{\lambda_\varepsilon} \mathcal{L}
\end{align}

Importantly, the choice of the reweighting coefficient doesn't depend on the parameterization of the model. We can parameterize the model throgh prediction $\hat{x}$, $\hat{\varepsilon}$, or $\hat{v}$ with same objective function.

## Rewaighted ELBO for NFDM

### General case

In the general case, when the forward process defined as

\begin{align}
    z = F(\varepsilon, t, x),
\end{align}

there is not much we can do. We can rewaight the ELBO with the $\frac{1}{2 g^2}$ coefficient, which is a part of the ELBO, but I'm not sure if it will help. We can also try to rewaight the ELBO with the different $\lambda$ coefficients from VDM. However, since $F$ doesn't have any connections with the SNR, I don't now what such a reweighting can give us.

### Less general case

We can consider the case with a less general forward process

\begin{align}
    z = \alpha F(x, t) + \sigma G(x, t) \varepsilon
\end{align}

This is a Gaussian forward process and it does have a connection with the SNR function. Therefore, we can reweight the ELBO with the $\lambda$ coefficients from VDM. If $F=x$ and $G=1$, we will recover exactly the VDM case.

### NDM

We can simplify the forward process a bit more and consider the NDM case

\begin{align}
    z = \alpha F(x, t) + \sigma \varepsilon
\end{align}

For this case, the same logic applies. We can take the true ELBO and reweight it with the $\lambda$ coefficients from VDM. However, in this case we can even slightly simplify the calculations. Let's write down the ELBO for the NDM:

\begin{align}
    f^B
    &= f - \frac{g^2}{2} s \\
    &= \dot{\alpha} F + \alpha \dot{F} + \frac{\dot{\sigma}}{\sigma} (z - \alpha F) - \frac{g^2}{2} \frac{\alpha F - z}{\sigma^2} \\
    &= \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha - \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right) F + \alpha \dot{F} + \left( \frac{\dot{\sigma}}{\sigma} + \frac{g^2}{2} \frac{1}{\sigma^2} \right) z
\end{align}

\begin{align}
    \mathcal{L} 
    &= \frac{1}{2 g^2} \left\| f^B - \hat{f}^B \right\|_2^2 \\
    &= \frac{1}{2 g^2} \left\| \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha - \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right) \left( F - \hat{F} \right) + \alpha \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{1}{2} \left\| \frac{1}{g} \left( \dot{\alpha} - \frac{\dot{\sigma}}{\sigma} \alpha - \frac{g^2}{2} \frac{\alpha}{\sigma^2} \right) \left( F - \hat{F} \right) + \frac{\alpha}{g} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{1}{2} \left\| \sqrt{2 \lambda_x} \left( F - \hat{F} \right) + \frac{\alpha}{g} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{1}{2} \left\| \sqrt{2 \lambda_x} \left( F - \hat{F} \right) + \frac{\alpha}{\sigma} \frac{\sqrt{\dot{\gamma}}}{\dot{\gamma}} \frac{\sqrt{\eta}}{\eta} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{1}{2} \left\| e^{-\frac{\gamma}{2}} \sqrt{\dot{\gamma}} \frac{ 1 + \eta }{2} \frac{\sqrt{\eta}}{\eta} \left( F - \hat{F} \right) + e^{-\frac{\gamma}{2}} \frac{\sqrt{\dot{\gamma}}}{\dot{\gamma}} \frac{\sqrt{\eta}}{\eta} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{1}{\eta} \left\| \frac{ 1 + \eta }{2} \left( F - \hat{F} \right) + \frac{1}{\dot{\gamma}} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \lambda_F \left\| \frac{ 1 + \eta }{2} \left( F - \hat{F} \right) + \frac{1}{\dot{\gamma}} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2, \quad \text{where} \quad \lambda_F = \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{1}{\eta}
\end{align}

Importantly, $\eta = 1$ doesn't necessarily minimises the ELBO in this case.

Therefore, if we want to train the NDM with $\mathcal{L}_x$ objective, we can do it as follows

\begin{align}
    \mathcal{L}_x
    &= \frac{1}{\lambda_x} \mathcal{L} \\
    &= \frac{\lambda_F}{\lambda_x} \left\| \frac{ 1 + \eta }{2} \left( F - \hat{F} \right) + \frac{1}{\dot{\gamma}} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{ \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{1}{\eta} }{ \frac{1}{2} e^{-\gamma} \dot{\gamma} \frac{ \left( 1 + \eta \right)^2 }{2^2 \eta} } \left\| \frac{ 1 + \eta }{2} \left( F - \hat{F} \right) + \frac{1}{\dot{\gamma}} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2 \\
    &= \frac{ 2^2 }{ \left( 1 + \eta \right)^2 } \left\| \frac{ 1 + \eta }{2} \left( F - \hat{F} \right) + \frac{1}{\dot{\gamma}} \left( \dot{F} - \dot{\hat{F}} \right) \right\|_2^2
\end{align}

## Implementation

### Imports and utils

In [422]:
from abc import ABC, abstractmethod
from typing import Callable, Optional

import numpy as np

import torch
from torch import nn, Tensor
from torch.nn import functional as F
import torch.distributions as D

import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [423]:
class Net(nn.Module):
    def __init__(self, in_dim: int, out_dim: int):
        super().__init__()

        self.net = nn.Sequential(
            nn.Linear(in_dim, 64),
            nn.SELU(),
            nn.Linear(64, 64),
            nn.SELU(),
            nn.Linear(64, 64),
            nn.SELU(),
            nn.Linear(64, 64),
            nn.SELU(),
            nn.Linear(64, out_dim),
        )

    def forward(self, t: Tensor) -> Tensor:
        return self.net(t)

In [424]:
@torch.no_grad()
def solve_sde(
        sde: Callable[[Tensor, Tensor], tuple[Tensor, Tensor]],
        z: Tensor,
        ts: float,
        tf: float,
        n_steps: int,
        show_pbar: bool=False
):
    bs = z.shape[0]

    t_steps = torch.linspace(ts, tf, n_steps + 1)
    dt = (tf - ts) / n_steps
    dt_2 = abs(dt) ** 0.5

    path = [z]
    pbar = tqdm if show_pbar else (lambda a: a)
    for t in pbar(t_steps[:-1]):
        t = t.expand(bs, 1)

        f, g = sde(z, t)

        w = torch.randn_like(z)
        z = z + f * dt + g * w * dt_2

        path.append(z)

    return z, (t_steps, torch.stack(path))

In [425]:
class TimeSampler(nn.Module, ABC):
    def __init__(self, salt_fraction: Optional[int] = None):
        super().__init__()

        self._salt_fraction = salt_fraction

    @abstractmethod
    def prob(self, t: Tensor) -> Tensor:
        raise NotImplementedError

    @abstractmethod
    def sample(self, bs: int) -> Tensor:
        raise NotImplementedError

    def loss(self, loss: Tensor, t: Tensor) -> Tensor:
        """
        In terms of minimization of the variance, this loss is not quite correct. Firstly, in lit module,
        we detach t and loss. Theoretically we should differentiate end-to-end through loss to obtain
        the true gradient w.r.t. parameters of the proposal distribution. However, to do this, we must
        differentiate through the training step second time just to optimize the proposal distribution,
        which is too expensive. Therefore, we detach t and loss and work with biased gradient. Secondly,
        we should take into account the salting, which we don't.
        """

        p = self.prob(t)

        l2 = loss ** 2
        p2 = p ** 2

        return l2 / p2

    def forward(self, bs: int) -> tuple[Tensor, Tensor]:
        t = self.sample(bs)

        dtype = t.dtype
        device = t.device

        if self._salt_fraction is not None:
            assert bs % self._salt_fraction == 0

            bs2 = bs // self._salt_fraction
            bs1 = bs - bs2

            un = D.Uniform(
                torch.tensor([0.], dtype=dtype, device=device),
                torch.tensor([1.], dtype=dtype, device=device)
            )
            u = un.sample(torch.Size((bs2,)))

            t = torch.cat([t[:bs1], u], dim=0)

            p = self.prob(t)

            k = 1 / self._salt_fraction
            p = p * (1 - k) + k
        else:
            p = self.prob(t)

        return t, p


class UniformSampler(TimeSampler):
    def __init__(self, salt_fraction: Optional[int] = None):
        super().__init__(salt_fraction)

        self.register_buffer("_l", torch.tensor(0.))
        self.register_buffer("_r", torch.tensor(1.))

    @property
    def _u(self) -> D.Uniform:
        return D.Uniform(self._l, self._r)

    def prob(self, t: Tensor) -> Tensor:
        return self._u.log_prob(t).squeeze(dim=1).exp()

    def sample(self, bs: int) -> Tensor:
        return self._u.sample(torch.Size((bs, 1)))


class BucketSampler(TimeSampler):
    def __init__(self, n: int = 100, salt_fraction: Optional[int] = None):
        super().__init__(salt_fraction)

        self._logits = nn.Parameter(torch.ones(n))

    @property
    @abstractmethod
    def _bucket_prob(self) -> Tensor:
        raise NotImplementedError

    @property
    @abstractmethod
    def _bucket_width(self) -> Tensor:
        raise NotImplementedError

    @property
    def _bucket_height(self) -> Tensor:
        return self._bucket_prob / self._bucket_width

    @property
    def _bucket_bounds(self) -> tuple[Tensor, Tensor]:
        w = self._bucket_width

        dtype = w.dtype
        device = w.device

        b_r = torch.cumsum(w, dim=0)
        b_l = torch.cat([torch.zeros(1, dtype=dtype, device=device), b_r[:-1]])

        return b_l, b_r

    def prob(self, t: Tensor) -> Tensor:
        t = t.flatten()

        t, ids_t = torch.sort(t)
        inv_ids_t = torch.argsort(ids_t)

        b_l, _ = self._bucket_bounds

        ids_p = torch.searchsorted(b_l, t, right=True) - 1

        p = self._bucket_height
        p = torch.index_select(p, 0, ids_p)
        p = torch.index_select(p, 0, inv_ids_t)

        return p

    def sample(self, bs: int) -> Tensor:
        b_p = self._bucket_prob
        b_l, b_r = self._bucket_bounds

        dtype = b_p.dtype
        device = b_p.device

        cat = D.Categorical(b_p)
        ids = cat.sample(torch.Size((bs,)))

        un = D.Uniform(
            torch.tensor(0., dtype=dtype, device=device),
            torch.tensor(1., dtype=dtype, device=device)
        )
        u = un.sample(torch.Size((bs,)))

        t = torch.index_select(b_l, 0, ids) + torch.index_select(b_r - b_l, 0, ids) * u
        t = t[:, None]

        return t


class UniformBucketSampler(BucketSampler):
    @property
    def _bucket_prob(self) -> Tensor:
        logits = self._logits
        logits = torch.clamp(logits, min=-10, max=10)

        return torch.softmax(logits, dim=0)

    @property
    def _bucket_width(self) -> Tensor:
        logits = self._logits
        dtype = logits.dtype
        device = logits.device
        n = logits.shape[0]
        return torch.ones(n, dtype=dtype, device=device) / n

In [426]:
def viz_2d_data(data: Tensor):
    plt.scatter(data[:, 0], data[:, 1], s=1)
    plt.axis("scaled")
    plt.show()

In [427]:
def viz_2d_path(t_steps: Tensor, path: Tensor, n_lines: int=-1, color: str | None=None):
    plt.figure(figsize=(12, 12))
    plt.scatter(15 + path[0, :, 0], path[0, :, 1], s=1)
    plt.scatter(path[-1, :, 0], path[-1, :, 1], s=1)
    plt.plot(15 * t_steps[:, None] + path[:, :n_lines, 0],
             path[:, :n_lines, 1],
             color=color, alpha=0.5)
    plt.axis("scaled")
    plt.tick_params(left=False, labelleft=False,
                    bottom=False, labelbottom=False)
    plt.show()

In [ ]:
def gen_data(n: int):
    scale = 4.
    centers = torch.tensor([
        [1, 0],
        [-1, 0],
        [0, 1],
        [0, -1],
        [1. / np.sqrt(2), 1. / np.sqrt(2)],
        [1. / np.sqrt(2), -1. / np.sqrt(2)],
        [-1. / np.sqrt(2), 1. / np.sqrt(2)],
        [-1. / np.sqrt(2), -1. / np.sqrt(2)]
    ], dtype=torch.float32)
    centers = scale * centers

    x = torch.randn(n, 2)
    x = 0.5 * x

    center_ids = torch.randint(0, 8, (n,))
    x = x + centers[center_ids]

    x = x / 2 ** 0.5

    return x

# generates data of dimension 2
data = gen_data(10)
print(data)

### Forward process

In [429]:
def jvp(f, x, v):
    return torch.autograd.functional.jvp(
        f, x, v,
        create_graph=torch.is_grad_enabled()
    )


def t_dir(f, t):
    return jvp(f, t, torch.ones_like(t))

In [430]:
class AffineTransform(nn.Module, ABC):
    @abstractmethod
    def get_m_s(self, x: Tensor, t: Tensor) -> tuple[Tensor, Tensor]:
        raise NotImplementedError

    def forward(self, x: Tensor, t: Tensor) -> tuple[tuple[Tensor, Tensor], tuple[Tensor, Tensor]]:
        def f(t_in):
            return self.get_m_s(x, t_in)

        return t_dir(f, t)

In [431]:
class AffineTransformID(AffineTransform):
    @staticmethod
    def get_m_s(x, t):
        m = x
        s = torch.ones_like(x)
        return m, s

    @staticmethod
    def forward(x, t): # this is VDM. 
        m, s = AffineTransformID.get_m_s(x, t)

        dm = torch.zeros_like(x)
        ds = torch.zeros_like(x)

        return (m, s), (dm, ds)


class AffineTransformHalfNeural(AffineTransform):
    def __init__(self, d: int):
        super().__init__()

        self.net = Net(d, d)

    def get_m_s(self, x, t):
        #x_t = torch.cat([x, t], dim=1)
        m = self.net(x)

        #m = x + t * m
        s = torch.ones_like(x)

        return m, s

In [432]:
class Gamma(nn.Module, ABC):
    @staticmethod
    def alpha_2(g):
        return torch.sigmoid(-g)

    @staticmethod
    def sigma_2(g):
        return torch.sigmoid(g)

    @abstractmethod
    def get_gamma(self, t: Tensor) -> Tensor:
        raise NotImplementedError

    def forward(self, t: Tensor) -> tuple[Tensor, Tensor]:
        return t_dir(self.get_gamma, t)

In [433]:
class GammaLinear(Gamma):
    @staticmethod
    def get_gamma(t):
        return -10 + 20 * t

    @staticmethod
    def forward(t):
        g = GammaLinear.get_gamma(t)
        dg = torch.ones_like(t) * 20
        return g, dg

In [434]:
class PosLinear(nn.Linear):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.sp = nn.Softplus()

    def forward(self, x: Tensor) -> Tensor:
        weight = self.sp(self.weight)
        bias = self.bias
        return F.linear(x, weight, bias)


class GammaVDM(Gamma):
    def __init__(self):
        super().__init__()

        self.fc1 = PosLinear(1, 1)
        self.fc2 = PosLinear(1, 1024)
        self.fc3 = PosLinear(1024, 1)

    def get_unnorm_gamma(self, x):
        x = self.fc1(x)

        y = self.fc2(x)
        y = torch.sigmoid(y)
        y = self.fc3(y)

        return x + y

    def get_gamma(self, t):
        x_0 = torch.zeros(1, 1)
        x_1 = torch.ones(1, 1)
        y_0 = torch.ones(1, 1) * (-10)
        y_1 = torch.ones(1, 1) * 10 #flag
        y_gap = y_1 - y_0

        x_adj = torch.cat([x_0, x_1, t], dim=0)
        y_adj = self.get_unnorm_gamma(x_adj)
        yo_0, yo_1, yo = y_adj[:1], y_adj[1:2], y_adj[2:]

        y = y_0 + (y_1 - y_0) * (yo - yo_0) / (yo_1 - yo_0)

        return y

In [435]:
class GammaMuLAN(Gamma):
    #implement get_gamma and forward methods

    def __init__(self, config):
        super().__init__()
        self.config = config
        
        self.gamma_shape = config.gamma_shape
        self.n_features = config.seq_len * config.embedding_dim #what does this reflect? -> n flattend features 
        self.min_gamma = self.config.gamma_min
        self.max_minus_min_gamma = self.config.gamma_max - self.config.gamma_min
        self.grad_min_epsilon = 0.001



        #self.l1 = nn.Linear(self.n_features, self.n_features)
        #if I want the noise injection to not depend on the input, more like vdm, since input dependence will be injected through ndm function transformation, I can just make the input of
        #l1 to be 1. which means the input is just the time t. That is not fully equal to MuLAN since there the a, b, d are dependent only on input not on t.
        #that also mean I will still need to implement the get_gamma method, but not the forward method, we need tdir still. 
        self.l1 = nn.Linear(2, self.n_features)
        self.l2 = nn.Linear(self.n_features, self.n_features)
        self.l3_a = nn.Linear(self.n_features, self.n_features)
        self.l3_b = nn.Linear(self.n_features, self.n_features)
        self.l3_c = nn.Linear(self.n_features, self.n_features)

    def _eval_polynomial(self, a, b, c, t):
        # Polynomial evaluation
        polynomial = (
            (a ** 2) * (t ** 5) / 5.0
            + (b ** 2 + 2 * a * c) * (t ** 3) / 3.0
            + a * b * (t ** 4) / 2.0
            + b * c * (t ** 2)
            + (c ** 2 + self.grad_min_epsilon) * t)
        
        scale = ((a ** 2) / 5.0
                 + (b ** 2 + 2 * a * c) / 3.0
                 + a * b / 2.0
                 + b * c
                 + (c ** 2 + self.grad_min_epsilon))

        return self.min_gamma + self.max_minus_min_gamma * polynomial / scale
    
    def _grad_t(self, a, b, c, t):
        # derivative = (at^2 + bt + c)^2
        polynomial = (
        (a ** 2) * (t ** 4)
        + (b ** 2 + 2 * a * c) * (t ** 2)
        + a * b * (t ** 3) * 2.0
        + b * c * t * 2
        + (c ** 2 + self.grad_min_epsilon))
        
        scale = ((a ** 2) / 5.0
                + (b ** 2 + 2 * a * c) / 3.0
                + a * b / 2.0
                + b * c
                + (c ** 2 + self.grad_min_epsilon))

        return self.max_minus_min_gamma * polynomial / scale

    def _compute_coefficients(self, x):
        _h = torch.nn.functional.silu(self.l1(x))
        _h = torch.nn.functional.silu(self.l2(_h))
        a = self.l3_a(_h)
        b = self.l3_b(_h)
        c = 1e-3 + torch.nn.functional.softplus(self.l3_c(_h))
        #print(a,b,c)
        return a, b, c
    
    def get_gamma(self, t, x):
        a, b, c = self._compute_coefficients(x)
        gamma_flat = self._eval_polynomial(a, b, c, t)
        #shape should be bs=t.shape[0], gamma_shape
        #how do I append a value to the shape though?
        
        gamma = gamma_flat.view(-1, *self.gamma_shape)
        #print(gamma.shape, "gamma shape")
        return gamma
    
    def forward(self, t, x):
        a, b, c = self._compute_coefficients(x)
        dg = self._grad_t(a, b, c, t)
        dg = dg.clamp(min=self.grad_min_epsilon)
        return self.get_gamma(t, x), dg

    #def forward(self, t):
    #    gamma, dgamma = t_dir(self.get_gamma, t)
    #    #dgamma = torch.clamp(dgamma, min=self.grad_min_epsilon)
    #    return gamma, dgamma
    
    """
    @staticmethod
    def sigma_2(g):

        vector = torch.sigmoid(g)
        sigma_2 = torch.diagonal
    """
    

from types import SimpleNamespace

mulan_config = SimpleNamespace(
                gamma_shape=(1,),
                seq_len= 1,
                embedding_dim= 1,
                gamma_min= -13.3,
                gamma_max= 5.0
                )

gamma_here = GammaMuLAN(mulan_config)
    

In [436]:
class VolatilityEta(nn.Module, ABC):
    @abstractmethod
    def forward(self, t: Tensor) -> Tensor:
        raise NotImplementedError

In [437]:
class VolatilityEtaOne(nn.Module):
    def forward(self, t):
        return torch.ones_like(t)


class VolatilityEtaNeural(nn.Module, ABC):
    def __init__(self):
        super().__init__()

        self.net = Net(1, 1)
        self.sp = nn.Softplus()

    def forward(self, t):
        return self.sp(self.net(t))

class VolatilityEtaOneNew(nn.Module):
    def forward(self, t):
        return torch.ones(t.size(0), 2)
#vol = VolatilityEtaOneNew()
#print(vol(torch.tensor([1,2,4,5,6,7,54,2,3,4,5,6])))

### Reverse process

In [438]:
class Predictor(nn.Module):
    def __init__(self, d: int):
        super().__init__()

        self.net = Net(d + 1, d)

    def forward(self, z, t):
        z_t = torch.cat([z, t], dim=1)
        x = self.net(z_t)

        x = (1 - t) * z + (t + 0.01) * x

        return x

### Neural diffusion

In [439]:
class NeuralDiffusion(nn.Module):
    def __init__(self, transform: AffineTransform, gamma: Gamma, vol_eta: VolatilityEta, pred: Predictor, VAE=False):
        super().__init__()

        self.transform = transform
        self.gamma = gamma
        self.vol_eta = vol_eta
        self.pred = pred
        #self.scalar = nn.Parameter(torch.tensor([1.0]))
        
        self.VAE = VAE
        if VAE:
            self.model = Net(2, 4)
            print(self.model.requires_grad_, "model requires grad")
        else:
            self.model = Net(2, 2)

    
    def forward(self, x: Tensor, t: Tensor):
        #print("fakka")
        #print(torch.is_grad_enabled(), "is grad enabled")
        eps = torch.randn_like(x)

        # Check if eps contains NaNs or Infs
        if torch.any(torch.isnan(eps)) or torch.any(torch.isinf(eps)):
            print(f"NaN or Inf detected in eps: {eps}")

        if self.VAE:
            mean, log_std = self.model(x).chunk(2, dim=1)
            std = torch.exp(log_std)
            context = mean + std * torch.randn_like(std)
        else:
            context = self.model(x)

        gamma, d_gamma = self.gamma(t, context)

        # Check gamma and d_gamma for NaNs and Infs
        if torch.any(torch.isnan(gamma)) or torch.any(torch.isinf(gamma)):
            print(f"NaN or Inf detected in gamma: {gamma}")
            print(gamma[torch.isinf(gamma)])
            print(gamma[torch.isnan(gamma)])

        if torch.any(torch.isnan(d_gamma)) or torch.any(torch.isinf(d_gamma)): # or torch.any(d_gamma < 0.000001):
            print(f"NaN or Inf detected in d_gamma: {d_gamma}")
        
        alpha = self.gamma.alpha_2(gamma) ** 0.5
        sigma = self.gamma.sigma_2(gamma) ** 0.5

        # Check alpha and sigma for NaNs or Infs
        if torch.any(torch.isnan(alpha)) or torch.any(torch.isinf(alpha)):
            print(f"NaN or Inf detected in alpha: {alpha}")
        if torch.any(torch.isnan(sigma)) or torch.any(torch.isinf(sigma)):
            print(f"NaN or Inf detected in sigma: {sigma}")

        (m, _), (d_m, _) = self.transform(x, t)

        # Check m, d_m for NaNs or Infs
        if torch.any(torch.isnan(m)) or torch.any(torch.isinf(m)):
            print(f"NaN or Inf detected in m: {m}")
        if torch.any(torch.isnan(d_m)) or torch.any(torch.isinf(d_m)):
            print(f"NaN or Inf detected in d_m: {d_m}")

        eta = self.vol_eta(t)

        # Check eta for NaNs or Infs
        if torch.any(torch.isnan(eta)) or torch.any(torch.isinf(eta)):
            print(f"NaN or Inf detected in eta: {eta}")

        z = alpha * m + sigma * eps

        # Check z for NaNs or Infs
        if torch.any(torch.isnan(z)) or torch.any(torch.isinf(z)):
            print(f"NaN or Inf detected in z: {z}")

        x_ = self.pred(z, t)
        #print(x_.requires_grad, "x_ requires grad")

        # Check x_ for NaNs or Infs
        if torch.any(torch.isnan(x_)) or torch.any(torch.isinf(x_)):
            print(f"NaN or Inf detected in x_: {x_}")

        (m_, _), (d_m_, _) = self.transform(x_, t)

        # Check m_, d_m_ for NaNs or Infs
        if torch.any(torch.isnan(m_)) or torch.any(torch.isinf(m_)):
            print(f"NaN or Inf detected in m_: {m_}")
        if torch.any(torch.isnan(d_m_)) or torch.any(torch.isinf(d_m_)):
            print(f"NaN or Inf detected in d_m_: {d_m_}")

        # ELBO weighting
        lmbd = 0.5 * torch.exp(-gamma) * d_gamma / eta
        #lmbd = 0.5 * torch.exp(-self.scalar * gamma) * d_gamma / eta
        #lmbd = 0.5 * 8 / eta
        #print(lmbd)

        
        #lmbd = 0.5 * d_gamma / eta
        #print("exp gamma is : ", torch.exp(-gamma))
        #print("d_gamma is : ", d_gamma)

        # Check lmbd for NaNs or Infs
        if torch.any(torch.isnan(lmbd)) or torch.any(torch.isinf(lmbd)):
            print(f"NaN or Inf detected in lmbd: {lmbd}")

        # L_x weighting (optional, commented out)
        #lmbd = (4 / (1 + eta) ** 2) 
        #print("lmbd is: ", lmbd) #should just be 1, it is
        one_over_dgamma = torch.clamp(1 / (d_gamma), max=10000) 
        if torch.any(one_over_dgamma == 10000):
            print("clamped")
        
        #print("one_over_dgamma is: ", one_over_dgamma) #should just be 1, it is

        loss = (1 + eta) / 2 * (m - m_) + one_over_dgamma * (d_m - d_m_)

        #print(d_gamma[d_gamma < 0.000001], "dgamma")
        #print(one_over_dgamma[d_gamma < 0.000001])
        #print(d_m-d_m_, "should be zero")
        #print(one_over_dgamma*(d_m - d_m_), "should be zero")
       
        #print("unscaled loss is: ", loss)

        # Check intermediate loss for NaNs or Infs
        if torch.any(torch.isnan(loss)) or torch.any(torch.isinf(loss)):
            print(f"NaN or Inf detected in loss before squaring: {loss}")

        loss = loss ** 2

        # Check squared loss for NaNs or Infs
        if torch.any(torch.isnan(loss)) or torch.any(torch.isinf(loss)):
            print(f"NaN or Inf detected in squared loss: {loss}")

        # Stabilises training
        #loss_x = (1 + eta) ** 2 / 4 * (x - x_) ** 2

        # Check loss_x for NaNs or Infs
        #if torch.any(torch.isnan(loss_x)) or torch.any(torch.isinf(loss_x)):
        #    print(f"NaN or Inf detected in loss_x: {loss_x}")

        #loss = 0.5 * loss + 0.5 * loss_x

        # Check final loss before applying lambda for NaNs or Infs
        if torch.any(torch.isnan(loss)) or torch.any(torch.isinf(loss)):
            print(f"NaN or Inf detected in final loss before lambda: {loss}")

        loss = lmbd * loss 

        # Check final loss after applying lambda for NaNs or Infs
        if torch.any(torch.isnan(loss)) or torch.any(torch.isinf(loss)):
            print(f"NaN or Inf detected in final loss: {loss}")

        # Optionally, you can add assertions here to ensure that no NaN or Inf values propagate.
        assert not torch.any(torch.isnan(loss)), f"NaN detected in final loss: {loss}"
        assert not torch.any(torch.isinf(loss)), f"Inf detected in final loss: {loss}"

        loss = loss.sum(dim=1)
        #print(loss.shape)

        # Check final loss sum for NaNs or Infs
        if torch.any(torch.isnan(loss)) or torch.any(torch.isinf(loss)):
            print(f"NaN or Inf detected in loss after summing: {loss}")

        
        #if we are using VAE we need to also compute the prior VAE loss 
        if self.VAE:
            #print(log_std.shape)
            #print(mean.shape)
            KLD = -0.5 * torch.sum(1 + (2*log_std) - mean**2 - (2*log_std).exp(), dim=-1)
            #print(KLD.shape)
            #print(loss.shape)
            #print(KLD.requires_grad, "KLD requires grad")

        #check KLD for NaNs or Infs
        if torch.any(torch.isnan(KLD)) or torch.any(torch.isinf(KLD)):
            print(f"NaN or Inf detected in KLD: {KLD}")
            print(KLD)
            print(KLD[torch.isinf(KLD)])
            print(KLD[torch.isnan(KLD)])
            print(KLD[KLD > 10000])
        
        #print(loss.mean(), "loss mean")
        return loss, KLD


### Training

In [ ]:
data_sample = gen_data(2 ** 12)

viz_2d_data(data_sample)

In [441]:
def train(model, time_sampler, gen_data_f):
    iter = 10 ** 5
    bs = 2 ** 10

    #set lr higher for model.gamma than for the rest using param group

    optim = torch.optim.Adam([*model.parameters(), *time_sampler.parameters()], lr=1e-2)

    pbar = trange(iter)
    for i in pbar:
        x = gen_data_f(bs)

        t, p = time_sampler(bs=bs)
        t, p = t.detach(), p.detach()

        loss, KLD = model(x, t)
        loss = loss + KLD
        loss = loss / p + time_sampler.loss(loss.detach(), t)

        

        # plt.scatter(t[:, 0], loss.detach().numpy(), s=1)
        # plt.scatter(t[:, 0], p.detach().numpy(), s=1)
        # plt.show()

        loss = loss.mean()

        if (i + 1) % 100 == 0:
            pbar.set_description(f"{loss.item():.4f}")

        #print(loss)
        optim.zero_grad()
        loss.backward()
        optim.step()

In [446]:
mulan_config = SimpleNamespace(
                gamma_shape=(1,),
                seq_len= 1,
                embedding_dim= 1,#2
                gamma_min= -10, #-13.3
                gamma_max= 10, # 5
                learn_tau=False,
                learn_delta=False
            )
torch.set_grad_enabled(True)           
transform = AffineTransformID()
#transform = AffineTransformHalfNeural(d=2)

#gamma = GammaLinear()
#gamma = GammaVDM()
#gamma = GammaBad()
gamma = GammaMuLAN(mulan_config)


vol_eta = VolatilityEtaOne()
#vol_eta = VolatilityEtaNeural()

pred = Predictor(d=2)

ndm = NeuralDiffusion(transform, gamma, vol_eta, pred, VAE=True)#, VAE=True)
ndm.train()

time_sampler = UniformBucketSampler()
#time_sampler = UniformSampler()

train(model=ndm, time_sampler=time_sampler, gen_data_f=gen_data)
print("hallo")

<bound method Module.requires_grad_ of Net(
  (net): Sequential(
    (0): Linear(in_features=2, out_features=64, bias=True)
    (1): SELU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): SELU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): SELU()
    (6): Linear(in_features=64, out_features=64, bias=True)
    (7): SELU()
    (8): Linear(in_features=64, out_features=4, bias=True)
  )
)> model requires grad


239.8645:   5%|▌         | 5453/100000 [00:58<16:55, 93.07it/s] 


ValueError: Expected parameter probs (Tensor of shape (100,)) of distribution Categorical(probs: torch.Size([100])) to satisfy the constraint Simplex(), but found invalid values:
tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
        nan, nan, nan, nan], grad_fn=<DivBackward0>)

In [ ]:
# plot the distibution of the VAE latent space
def plot_vae_latent_space(model, data):
    model.eval()
    with torch.no_grad():
        mean, log_std = model.model(data).chunk(2, dim=1)
        std = torch.exp(log_std)
        context = mean + std * torch.randn_like(std)

    plt.figure(figsize=(8, 8))
    plt.scatter(context[:, 0].numpy(), context[:, 1].numpy(), s=1)
    plt.axis("scaled")
    plt.title("VAE Latent Space")
    plt.show()

plot_vae_latent_space(ndm, gen_data(2 ** 8))

In [ ]:
def plot_pdf(sampler: BucketSampler, num_points=1000):
    with torch.no_grad():
        # Generate evenly spaced t values
        t_values = torch.linspace(0, 1, num_points).view(-1, 1)  # Shape: (num_points, 1)
        
        # Compute p(t) at these points
        p_values = sampler.prob(t_values).cpu().numpy().flatten()
        t_values = t_values.cpu().numpy().flatten()

        # Compute the integral using the trapezoidal rule
        dt = 1 / (num_points - 1)  # Step size
        integral = np.trapz(p_values, t_values)  # Numerical integration

        # Plot the learned PDF
        plt.plot(t_values, p_values, label=f"Learned PDF (∫p(t)dt ≈ {integral:.4f})", color="b", linewidth=2)
        plt.xlabel("t")
        plt.ylabel("p(t)")
        plt.title("Learned Probability Density Function")
        plt.legend()
        plt.grid(True)
        plt.show()

        # Print the integral value
        print(f"Estimated integral of p(t) over [0,1]: {integral:.4f}")
        if abs(integral - 1) > 0.05:
            print("⚠️ Warning: PDF is not properly normalized!")
        else:
            print("✅ PDF is properly normalized.")

plot_pdf(time_sampler)

#isnt this the wrong way arround? loss should be higher for high t not low t, and gamma isnt focussed on any particular part yet 

In [ ]:
with torch.no_grad():
    t = torch.linspace(0, 1, 300)[:, None]

    gamma_og = GammaMuLAN(mulan_config)
    x = gen_data(300)

    #g_og, d_gamma = gamma_og(t, x)

    g, _ = gamma(t, x)


    plt.plot(t, g)
    #plt.plot(t, g_og)
    plt.legend(["Learned", "Original"])
    plt.show()

    #print(d_gamma)



In [ ]:
#also print the loss for these same values of 10
#x = gen_data(1).repeat(1000, 1)
x = gen_data(1000)
t_points = torch.linspace(0, 1, 1000)[:, None]

losses = []
for t in t_points:
    t = t.expand(x.shape[0], 1)
    #print(t.shape)
    #print(x.shape)
    #print(t)
    loss = ndm(x, t)
    avg_loss = loss.mean()
    losses.append(avg_loss.item())


print(losses)
loss = torch.tensor(losses)
plt.plot(t_points, loss.detach().numpy())
plt.show() 

#same 

### Train MuLAN

\begin{align}
f^B = \dot{\alpha} F + \alpha \dot{F} + \frac{\dot{\sigma}}{\sigma} (z - \alpha F) - \frac{g^2}{2} \frac{\alpha F - z}{\sigma^2} \\
\end{align}

## Visualisation

### SDE Sampling

In [ ]:
bs = 2 ** 12

z = torch.randn(bs, 2)


def sde(z_in, t_in):
    
    x_ = pred(z_in, t_in)
    if ndm.VAE:
        context = torch.randn_like(z_in)
    else:
        context = ndm.model(x_)
    
    gmm, d_gmm = gamma(t_in, context)
    alpha_2 = gamma.alpha_2(gmm)
    sigma_2 = gamma.sigma_2(gmm)
    alpha = alpha_2 ** 0.5
    sigma = sigma_2 ** 0.5 #I added this 
    if torch.any(torch.isnan(gmm)) or torch.any(torch.isinf(gmm)):
        print(t_in, "t_in")
        print(x_, "x_")
        print(gmm[torch.isnan(gmm)], "gmm")
        print("gmm is nan or inf")
    #print(gmm, "gmm")
    #print(d_gmm, "d_gmm")
    #print(alpha_2, "alpha_2")
    #print(sigma_2, "sigma_2")
    #print(alpha, "alpha")
    #print(sigma, "sigma")

    eta = vol_eta(t_in)

    g = (sigma_2 * d_gmm * eta) ** 0.5

    (m_, _), (d_m_, _) = transform(x_, t_in)

    #print(d_m_, "d_m")
    #print(sigma_2, "sigma2")
    #print(eta, "eta")
    #print(d_gmm, "d_gmm")
    #print(g, "g")

    #drift = -alpha * d_gmm * (1 + eta) / 2 * m_ + \
    #        alpha * d_m_ + \
    #        0.5 * d_gmm * (alpha_2 + eta) * z_in
    eps = (z_in - alpha * m_) / sigma
    alpha_prime = - d_gmm * 0.5 * alpha * (1- alpha_2) 
    sigma_prime = 0.5 * d_gmm * sigma * (1 - sigma_2)
    #dz = -alpha * d_gmm + alpha * d_m_ + sigma * d_gmm * eps
    dz = alpha_prime * m_ + alpha * d_m_ + sigma_prime * eps
    drift = dz - 0.5 * (g ** 2) * ((alpha * m_ - z_in) / sigma_2)

    #print(eps, "eps")
    #print(alpha_prime, "alpha_prime")
    #print(sigma_prime, "sigma_prime")
    #print(dz, "dz")
    #print(drift, "drift")
    if torch.any(torch.isnan(drift)) or torch.any(torch.isinf(drift)):
        print("drift is nan or inf")
    
    if torch.any(torch.isnan(g)) or torch.any(torch.isinf(g)):
        print("g is nan or inf")

    return drift, g


_, (t_steps, path) = solve_sde(sde=sde, z=z, ts=1, tf=0, n_steps=300)

print(path[-1])
viz_2d_path(t_steps, path, n_lines=16, color="blue")

### Star Diffusion

In [26]:
def denoised_fn(x, t):
    pass

In [ ]:
def predicting_part(prev_sample, t, denoised_fn):
    def process_xstart(x):
        if denoised_fn is not None:
            # print(denoised_fn)
            x = denoised_fn(x, t)
        if False:# clip_denoised:
            return x.clamp(-1, 1)
        return x
    
    out = {}
    x_ = pred(prev_sample, t) 
    
    x_start = process_xstart(x_)
    out["pred_xstart"] = x_start

    gmm, _ = gamma(t)
    alpha = gamma.alpha_2(gmm) ** 0.5
    sigma2 =  gamma.sigma_2(gmm)

    m, _ = transform.get_m_s(x_start, t)

    out["mean"] = alpha*m
    out["log_variance"] = torch.log(sigma2)


    noise = torch.randn_like(prev_sample)
    nonzero_mask = (
        (t != 0).float().view(-1, *([1] * (len(prev_sample.shape) - 1)))
    )  # no noise when t == 0
    sample = out["mean"] + nonzero_mask * torch.exp(0.5 * out["log_variance"]) * noise 
    return sample

@torch.no_grad()
def discrete_sampling_star(
        z: Tensor,
        ts: float,
        tf: float,
        n_steps: int,
        show_pbar: bool=True
):
    bs = z.shape[0]

    t_steps = torch.linspace(ts, tf, n_steps + 1)#[:-1]
    dt = (tf - ts) / n_steps
    dt_2 = abs(dt) ** 0.5

    path = [z]
    pbar = tqdm if show_pbar else (lambda a: a)
    for t in pbar(t_steps):
        t = t.expand(bs, 1)

        z = predicting_part(prev_sample=z, t=t, denoised_fn=None)

        path.append(z)

    return z, (t_steps, torch.stack(path[:-1]))


bs = 2 ** 12

z = torch.randn(bs, 2)

_, (t_steps, path) = discrete_sampling_star(z=z, ts=1, tf=0, n_steps=300)

print(path[-1])
viz_2d_path(t_steps, path, n_lines=16, color="blue")

### Correct Marginal Sampling

\begin{align}
    z_t = F(\varepsilon, t, x)
\end{align}

Then given z_t and t (n(0,1) noise and t=1), do:

1. x_0 prediction, $$x\_ = model.pred(z_t, t)$$
2. get epsilon by inverse big F, $$\varepsilon = \frac{(z - \alpha F) }{\sigma}$$
3. get epsilon s|t, 

\begin{align}
    \tilde{\varepsilon}_{s|t} = \sqrt{1- \tilde{\sigma}^2_{s|t}} \varepsilon + \tilde{\sigma}_{s|t} \tilde{\varepsilon}
\end{align}

where,

\begin{align}
    \tilde{\sigma}_{s|t} = \sigma_s^2 - \frac{SNR(t)}{SNR(s)}\sigma_s^2
\end{align}

and,

\begin{align}
 \tilde{\varepsilon} - N(0,1)? 
\end{align}
because otherwise no new noise would be injected


4. get z_s (next z) by feeding through the forward process. 
\begin{align}
    z = F(\tilde{\varepsilon}_{s|t}, s, x)
\end{align}

\begin{align}
    z = \alpha_s F + \sigma_s (\sqrt{1- \tilde{\sigma}^2_{s|t}} \varepsilon + \tilde{\sigma}_{s|t} \tilde{\varepsilon})
\end{align}

but that doesnt match the ndm appendix 1 I think? and it doesnt lead to a marginalized standard deviation of sigma_s, which would instead need,

\begin{align}
    z = \alpha_s F + (\sqrt{\sigma_s^2 - \tilde{\sigma}^2_{s|t}} \varepsilon + \tilde{\sigma}_{s|t} \tilde{\varepsilon})
\end{align}

which I suppose you could also write as 
\begin{align}
    \tilde{\varepsilon}_{s|t} = \frac{1}{\sigma^2_s}(\sqrt{\sigma^2_s- \tilde{\sigma}^2_{s|t}} \varepsilon + \tilde{\sigma}_{s|t} \tilde{\varepsilon})
\end{align}



In [ ]:
def get_next_marginal(prev_sample, t, s, denoised_fn):
    def process_xstart(x):
        if denoised_fn is not None:
            # print(denoised_fn)
            x = denoised_fn(x, t)
        if False:# clip_denoised:
            return x.clamp(-1, 1)
        return x
    
    #step 1 do prediction 
    x_ = pred(prev_sample, t) 
    
    x_start = process_xstart(x_)

    #step 2 get epsilon
    gmm, _ = gamma(t)
    alpha2 = gamma.alpha_2(gmm)
    sigma2 =  gamma.sigma_2(gmm)
    alpha = alpha2 ** 0.5
    sigma = sigma2 ** 0.5

    m_ , _ = transform.get_m_s(x_start, t)

    eps = (prev_sample - alpha * m_) / sigma

    #step 3 get epsilon s|t
    #we need stepsize for this?
    noise = torch.randn_like(prev_sample)
    gmm_s, _ = gamma(s)
    alpha2_s = gamma.alpha_2(gmm_s)
    sigma2_s =  gamma.sigma_2(gmm_s)
    alpha_s = alpha2_s ** 0.5
    sigma_s = sigma2_s ** 0.5
    
    m_s , _ = transform.get_m_s(x_start, s)

    #print(gmm_s)
    snr_t = (alpha2/sigma2).double()
    snr_s = (alpha2_s/sigma2_s).double()

    #sigma2_tilde_s_t = (1 -  (snr_t / snr_s)).float() #instead of casting back to float here we can alos cast back only after computing epsilon tilde st
    sigma2_tilde_s_t = -torch.expm1(gmm_s - gmm) #should be in 0-1
    #print(sigma2_tilde_s_t, "sigma2_tilde_s_t")

    #or option 3

    epsilon_tilde_s_t = torch.sqrt(1 - sigma2_tilde_s_t) * eps + (sigma2_tilde_s_t.sqrt()) * noise 

    #print("snr_t", snr_t[0])
    #print("snr_s", snr_s[0])
    #print("sigma", sigma2_tilde_s_t) #this should be positive always but isnt so im doing something wrong. 

    #step 4 get z_s
    sample = alpha_s * m_s + sigma_s * epsilon_tilde_s_t
    
    #if we want to match appendix 1 of ndm paper I think it should instead be
    #sample = alpha_s * m_s +  torch.sqrt(sigma2 - sigma2_tilde_s_t) * eps + (sigma2_tilde_s_t ** 0.5) * noise

    return sample

@torch.no_grad()
def discrete_sampling(
        z: Tensor,
        ts: float,
        tf: float,
        n_steps: int,
        show_pbar: bool=True
):
    bs = z.shape[0]

    t_steps = torch.linspace(ts, tf, n_steps + 1)#[:-1]
    dt = (tf - ts) / n_steps
    dt_2 = abs(dt) ** 0.5

    path = [z]
    pbar = tqdm if show_pbar else (lambda a: a)
    for t in pbar(t_steps[:-1]):
        t = t.expand(bs, 1)

        #I understand I am doing 2x-1 the number of needed forward pass through gamma now, Ill fix that before putting it into the actual code.        
        z = get_next_marginal(prev_sample=z, t=t, s=t+dt, denoised_fn=None)

        path.append(z)

    return z, (t_steps, torch.stack(path))


bs = 2 ** 12

z = torch.randn(bs, 2)

_, (t_steps, path) = discrete_sampling(z=z, ts=1, tf=0, n_steps=300)

print(path[-1])
viz_2d_path(t_steps, path, n_lines=16, color="blue")

### ODE sampling

that means we need just f(z,t,eps)|eps=F(z,t,)înverser. We have, \\
\begin{align}
    z = F(\varepsilon, t, x),
\end{align}
and specifically in the ndm/vdm case, \\
\begin{align}
    z = \alpha F(x, t) + \sigma \varepsilon
\end{align}
The ODE drift is given by \\
\begin{align}
    dz = f_{tilde}(z_t, t, x\_)dt
\end{align}
Where 
\begin{align}
    f_{tilde}(z_t, t, x\_) &= d/dt F(eps, t, x) \\
    &= alpha' * m + alpha * dm + sigma' * eps
\end{align}

\begin{align}
f &= \dot{\alpha} F + \alpha \dot{F} + \frac{\dot{\sigma}}{\sigma} (z - \alpha F) 
\end{align}


In [ ]:
bs = 2 ** 12

z = torch.randn(bs, 2)


def ode(z_in, t_in):
    gmm, d_gmm = gamma(t_in)
    alpha_2 = gamma.alpha_2(gmm)
    sigma_2 = gamma.sigma_2(gmm)
    alpha = alpha_2 ** 0.5
    sigma = sigma_2 ** 0.5

    #eta = vol_eta(t_in)

    x_ = pred(z_in, t_in)

    (m_, _), (d_m_, _) = transform(x_, t_in)

    eps = (z_in - alpha * m_) / sigma
    alpha_prime = - d_gmm * 0.5 * alpha * (1- alpha_2) 
    sigma_prime = 0.5 * d_gmm * sigma * (1 - sigma_2)
    #dz = -alpha * d_gmm + alpha * d_m_ + sigma * d_gmm * eps
    dz = alpha_prime * m_ + alpha * d_m_ + sigma_prime * eps
    #dz = -alpha_prime + alpha * d_m_ + sigma_prime * eps

    
    return dz, 0


_, (t_steps, path) = solve_sde(sde=ode, z=z, ts=1, tf=0, n_steps=300)

viz_2d_path(t_steps, path, n_lines=16, color="blue")

### Adaptive step size ODE sampling

In [ ]:
import torch
from torchdiffeq import odeint

def ode_adaptor(t, z):
    #print(t)
    t = t.expand(z.shape[0], 1)
    #print(z.shape)
    #print(t.shape) # bs x 2, why is this bs by 2??, needs to be bs x 1 where t moves from 1 to 0 over time
    drift, _ = ode(z, t)
    return drift

@torch.no_grad()
def integrate_ode_torch(z0, t_span=(1.0, 0.0), atol=1e-6, rtol=1e-7):
    # Here, we only provide the initial conditions (z0) and the time span (t_span)
    # The solver will automatically select adaptive time steps within the provided range
    t = torch.linspace(1.0, 0.0, 10000)  # Reverse time to go from 1 to 0
    # Solve the ODE using RK45 (dopri5 with adaptive step size)
    path = odeint(ode_adaptor, z0, t = t, method='dopri5', atol=atol, rtol=rtol)

     # The solver automatically handles the time steps
    final_state = path[-1]  # Last step of integration

    return final_state, (t, path)

bs = 2 ** 12
print(bs)

z = torch.randn(bs, 2)

final, (t_steps, path) = integrate_ode_torch(z)

print("steps taken", len(path))
print("steps", t_steps)

viz_2d_data(final.detach().numpy())
viz_2d_path(t_steps.detach().numpy(), path.detach().numpy(), n_lines=16, color="blue")

In [ ]:
with torch.no_grad():
    t = torch.linspace(0, 1, 300)[:, None]

    g, _ = gamma(t)
    alpha2 = gamma.alpha_2(g)
    sigma2 = gamma.sigma_2(g)
    snr = alpha2 / sigma2

    plt.plot(t, snr)
    plt.show()

#clearly t > s implies snr(t) < snr(s)

In [ ]:
with torch.no_grad():
    t = torch.linspace(0, 1, 300)[:, None]

    g, _ = gamma(t)

    plt.plot(g)
    plt.show()

In [ ]:
with torch.no_grad():
    t = torch.linspace(0, 1, 300)[:, None]
    gmm, dgamma = gamma(t)
    alpha_2 = gamma.alpha_2(gmm)
    sigma_2 = gamma.sigma_2(gmm)
    alpha = alpha_2 ** 0.5
    sigma = sigma_2 ** 0.5

    #also get alpha and sigma from the sqrt function and plot them in the same graph as the other alpha and sigma, so make three plots

    plt.plot(alpha)
    plt.legend(["alpha", "alpha_sqrt"])
    plt.show()
    plt.close()

    plt.plot(sigma) #flag2
    plt.legend(["sigma", "sigma_sqrt"])
    plt.show()
    plt.close()

    plt.plot(gmm)
    plt.legend(["gamma"])
    plt.show()
    plt.close()

    plt.plot(dgamma)
    print(dgamma)
    plt.legend(["dgamma"])
    plt.show()
    plt.close()
